In [1]:
import threading
import warnings
import os
import time
import json
import traceback
from apscheduler.schedulers.background import BackgroundScheduler
from flask import Flask, request, session, jsonify, render_template_string
from linebot import LineBotApi
from linebot.v3.webhook import WebhookHandler
from linebot.models import TextSendMessage, FlexSendMessage, ImageSendMessage
from linebot.deprecations import LineBotSdkDeprecatedIn30
# from utils.whisper import Transcriber
from utils.line_liff import main_html, success_page_html, error_page_html
from utils.config import model_size, device, compute_type, prompt, home_dir, channel_access_token, channel_secret
from utils.utils import MeetingBot_Utils
from utils.check_user_inactivity import UserInactivity

warnings.filterwarnings('ignore', category=LineBotSdkDeprecatedIn30)

app = Flask(__name__)
app.secret_key = 'chatbot'
scheduler = BackgroundScheduler()
scheduler.add_job(UserInactivity, 'interval', minutes=30)
scheduler.start()  

# @app.route("/audio/transcriptions", methods=["POST"])
# def _transcribe():
#     if 'file' not in request.files:
#         return jsonify({"message": "key裡面沒有file"}), 400
#     file = request.files['file']
#     if file.filename == '':
#         return jsonify({"message": "沒有選擇檔案"}), 400

#     try:
#         with Transcriber(model_size, device, compute_type, prompt) as stt:
#             audio = file.read()
#             transcriptions = [seg for seg in stt(audio)]
#             return jsonify({
#                 "verbatim": '\n'.join([item[0] for item in transcriptions]),
#                 "text": '，'.join([item[1] for item in transcriptions])
#             }), 200
#     except av.error.InvalidDataError:
#         return jsonify({"message": "檔案格式錯誤"}), 400
#     except Exception as e:
#         return jsonify({"message": str(e)}), 500  

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'GET':
        session['user_id'] = request.args.get('liff.state')[1:]
        
    elif request.method == 'POST':
        file = request.files['file']
        user_id = session.get('user_id')
        try:
            meetingbot_utils = MeetingBot_Utils(user_id)
            logger = meetingbot_utils.get_logger()  
            file_path = home_dir + f'/audio/{user_id}_{file.filename}'
            file.save(file_path)
            logger.info(file_path)
            audio_duration = meetingbot_utils.get_audio_duration(file_path)
            estimated_time = round((audio_duration / 3600) * 8)
            logger.info(f'音檔長度：{estimated_time} 分鐘')

            logger.info('音檔處理中...')
            threading.Thread(target=meetingbot_utils.process_file, args=(file_path, file, logger)).start()
            # executor = concurrent.futures.ThreadPoolExecutor(max_workers=10)
            # executor.submit(meetingbot_utils.process_file, file_path, file, logger)
            rendered_page = render_template_string(success_page_html)
            LineBotApi(channel_access_token).push_message(user_id, TextSendMessage(text=f'音檔已下載完畢，預計處理時間約 {estimated_time} 分鐘。'))            
            return rendered_page
        except Exception as e:
            print(f"檔案上傳失敗: {traceback.print_exc()}")
            return render_template_string(error_page_html)
    return render_template_string(main_html)

@app.route("/", methods=['POST'])
def linebot_stt():
    start_time  = time.time()
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    line_bot_api = LineBotApi(channel_access_token)
    handler = WebhookHandler(channel_secret)
    signature = request.headers['X-Line-Signature']
    handler.handle(body, signature)
    event = json_data['events'][0]
    replyToken = event['replyToken']
    user_id = event['source']['userId']
    msg_type = event['message']['type']
    user_messages = event['message'].get('text', '')
    id = event['message']['id']

    meetingbot_utils = MeetingBot_Utils(user_id)
    logger = meetingbot_utils.get_logger()  

    try:
        if msg_type == 'audio':
            line_bot_api.reply_message(replyToken, TextSendMessage(text='音檔下載中，請稍後...'))
            check_transcoding = True
            threading.Thread(target=meetingbot_utils.process_audio, args=(id, logger, check_transcoding)).start()
            
        elif msg_type == 'file':
            line_bot_api.reply_message(replyToken, TextSendMessage(text='音檔下載中，請稍後...'))
            check_transcoding = False
            threading.Thread(target=meetingbot_utils.process_audio, args=(id, logger, check_transcoding)).start()
            
            # executor = concurrent.futures.ThreadPoolExecutor(max_workers=10)
            # executor.submit(meetingbot_utils.process_audio, id, logger, check_transcoding=(msg_type == 'audio'))
            
        elif msg_type == 'text':
            if '上傳檔案' in user_messages:
                flex_message = {
                    "type": "bubble",
                    "body": {
                        "type": "box",
                        "layout": "vertical",
                        "contents": [
                            {
                                "type": "text",
                                "text": "上傳檔案",
                                "weight": "bold",
                                "size": "xl"
                            },
                            {
                                "type": "button",
                                "style": "primary",
                                "action": {
                                    "type": "uri",
                                    "label": "上傳",
                                    "uri": f"https://liff.line.me/2004718657-jZ83LpBd?{user_id}"
                                }
                            }
                        ]
                    }
                }
                line_bot_api.push_message(user_id, FlexSendMessage(alt_text="上傳檔案", contents=flex_message))

            elif '逐字稿' in user_messages:
                line_bot_api.reply_message(replyToken, TextSendMessage(text='產生逐字稿，請稍候...'))
                content = meetingbot_utils.gpt_generate(user_messages, logger)
                for segment in meetingbot_utils.split_text_by_length(content, max_length=5000):
                    line_bot_api.push_message(user_id, TextSendMessage(text=segment))

            elif '相關建議' in user_messages:
                line_bot_api.reply_message(replyToken, TextSendMessage(text='產生相關建議，請稍候...'))
                content = meetingbot_utils.gpt_generate(user_messages, logger)
                line_bot_api.push_message(user_id, TextSendMessage(text=content))
                            
            elif '會議摘要' in user_messages:
                line_bot_api.reply_message(replyToken, TextSendMessage(text='產生會議摘要，請稍候...'))
                content = meetingbot_utils.gpt_generate(user_messages, logger)
                line_bot_api.push_message(user_id, TextSendMessage(text=content))
               
            elif '心智圖' in user_messages:
                line_bot_api.reply_message(replyToken, TextSendMessage(text='產生心智圖中，請稍候...'))
                content = meetingbot_utils.gpt_generate('會議摘要', logger)
                if content is not None:
                    # meetingbot_utils.markmap_html(content, logger)
                    image = meetingbot_utils.markmap_to_png(content)
                    logger.info(f"上傳圖片的網址：{image}")
                    line_bot_api.push_message(user_id, ImageSendMessage(original_content_url=image, preview_image_url=image))
                else:
                    line_bot_api.push_message(user_id, TextSendMessage(text='請先輸入語音訊息或上傳檔案'))
                
            else:
                line_bot_api.reply_message(replyToken, TextSendMessage(text='目前只回答按鈕功能'))
        else:
            line_bot_api.reply_message(replyToken, TextSendMessage(text='目前只回答按鈕功能'))
        
        compute_time = time.time() - start_time
        logger.info(f'本次回覆時間：{compute_time:.2f} 秒')

    except Exception as e:
        logger.error(e)
        print(f"發生問題: {traceback.print_exc()}")
        line_bot_api.reply_message(replyToken, TextSendMessage(text='處理過程中出現了一些錯誤'))
    return 'Success'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

2024/07/30 11:53:10 AM - INFO : 上傳圖片的網址：https://i.imgur.com/BxvJajW.png
2024/07/30 11:53:10 AM - INFO : 本次回覆時間：27.20 秒
127.0.0.1 - - [30/Jul/2024 11:53:10] "POST / HTTP/1.1" 200 -
